<a href="https://colab.research.google.com/github/DhruvK278/Invoice-OCR/blob/main/invoice_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract
!sudo apt install tesseract-ocr

In [ ]:
import cv2
import matplotlib.pyplot as plt

# The file is now in the root directory of the Colab session
image_path = 'invoice-123.png'

# Read and display the image
try:
    image = cv2.imread(image_path)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Uploaded Invoice")
    plt.axis('off')
    plt.show()
except Exception as e:
    print(f"Error loading image. Make sure '{image_path}' is uploaded correctly.")
    print(e)